In [17]:
import os
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image

In [18]:
train_dataset_path = "../dataset/train/"
test_dataset_path = "../dataset/test/"

In [19]:
model1 = load_model('model1.h5')
model2 = load_model('model2.h5')
model3 = load_model('model3.h5')

In [20]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

In [21]:
train_generator = datagen.flow_from_directory(
    train_dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
)

test_generator = datagen.flow_from_directory(
    test_dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
)

Found 12000 images belonging to 4 classes.
Found 1805 images belonging to 4 classes.


In [22]:
class_names = ["Black Rot", "ESCA", "Healthy", "Leaf Blight"]

In [23]:
model1 = None
model2 = None
model3 = None

if model1 is None:
    model1 = load_model("model1.h5")
if model2 is None:
    model2 = load_model("model2.h5")
if model3 is None:
    model3 = load_model("model3.h5")

models = [model1, model2, model3]

In [24]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [25]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 26, 26, 128)      

In [26]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 26, 26, 128)      

In [27]:
testX, testY = next(test_generator)

In [28]:
def ensemble_predictions(models, testX):
    results = [model.predict(testX) for model in models]
    results = np.array(results)
    summed = np.sum(results, axis=0)
    return np.argmax(summed, axis=1)

In [29]:
ensemble_pred = ensemble_predictions(models, testX)

1/1 [==============================] - 0s 287ms/step


In [30]:
accuracy = np.mean(ensemble_pred == np.argmax(testY, axis=1))
print('Ensemble Accuracy: %.3f' % (accuracy * 100))

Ensemble Accuracy: 100.000


In [31]:
img_path = "../testing.png"
img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0

In [32]:
ensemble_pred = ensemble_predictions(models, img)

predicted_class = class_names[ensemble_pred[0]]
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 110ms/step
Predicted class: Healthy
